In [8]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.messages import SystemMessage,HumanMessage
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel,schema,Field
from typing import TypedDict,Annotated,Literal
from langgraph.checkpoint.memory import MemorySaver
import operator

In [3]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
llm = ChatOllama(
    model='qwen2:7b'
)


In [6]:
def chat_node(state: ChatState):

    messages = state['messages']

    response = llm.invoke(messages)

    return {'messages': [response]}

In [9]:
checkpoint = MemorySaver()
graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpoint)

In [10]:
thread_id = '1'

while True:

    user_message = input('Type here: ')

    print('User:', user_message)

    if user_message.strip().lower() in ['exit', 'quit', 'bye']:
        break

    config = {'configurable': {'thread_id': thread_id}}

    response = chatbot.invoke({'messages': [HumanMessage(content=user_message)]}, config=config)

    print('AI:', response['messages'][-1].content)

User: hi
AI: Hello! How can I assist you today?
User: my name is hriday
AI: Hello Hriday, nice to meet you! How can I help you today?
User: what is my name
AI: Your name is Hriday.
User: bye
